In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-xjtu/sic_6_1.npy
/kaggle/input/sample-xjtu/dic_3_4.npy
/kaggle/input/sample-xjtu/dic_6_1.npy
/kaggle/input/sample-xjtu/dic_3_2.npy
/kaggle/input/sample-xjtu/sic_3_4.npy
/kaggle/input/sample-xjtu/dic_3_3.npy
/kaggle/input/sample-xjtu/sic_3_6.npy
/kaggle/input/sample-xjtu/dic_1_1.npy
/kaggle/input/sample-xjtu/sic_1_1.npy
/kaggle/input/sample-xjtu/dic_3_5.npy
/kaggle/input/sample-xjtu/dic_3_1.npy
/kaggle/input/sample-xjtu/dic_3_6.npy
/kaggle/input/sample-xjtu/sic_3_1.npy
/kaggle/input/sample-xjtu/sic_3_2.npy
/kaggle/input/sample-xjtu/sic_3_3.npy
/kaggle/input/sample-xjtu/sic_3_5.npy


# 1 import required package

In [2]:
import time
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn import model_selection
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 2 get the index and features name

In [4]:
index = [0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 
         16, 19, 21, 22, 25, 27, 28, 29, 31, 32,
         34, 35, 39, 40, 41, 42, 46, 48, 49, 50, 
         51, 53, 55, 56, 57, 58, 60, 61, 63, 64, 
         70, 73, 74, 75, 77, 78, 79, 80, 82, 83, 
         84, 85, 89, 91, 92, 98, 100, 102, 105, 
         106, 107, 110, 111, 115, 116, 117, 119, 
         120, 122, 124, 131, 133, 135, 136, 138, 
         140, 141, 144, 145, 146, 147, 150, 151, 
         152, 155, 157, 159, 160, 161, 162, 163, 
         164, 165, 166, 170, 171, 173]

items = ['PLT', 'INR', 'D-Dimer', 'FIB', 'PT', 'HBA1c', 'HGB', 
         'Na', 'EO%', 'APOB', 'CK-MB', 'pO2', 'TBIL', 'Ca', 
         'TG', 'Cys-c', 'TP', 'CHOI', 'TCO2', 'gGlu', 'CO2CP', 
         'gHGB', 'PTA', 'NRBC%', 'ABE', 'MCV', 'CHE', 'Cl-', 
         'CaO2', 'PDW', 'α-HBDH', 'CRE', 'RDW-CV', 'WBC', 
         'P-LCR', 'GA%', 'K+', 'AMY', 'SOD', 'CK', 'TBA', 
         'MCH', 'UA', 'Mg', 'Lac', 'GLB', 'HDL', 'FCOHb', 
         'LDH', 'Glu', 'IG%', 'BUN', 'Cl', 'SpO2', 'PA-aDO2', 
         'FDP', 'p50', 'MetHb', 'APOA', 'DBIL', 'MCHC', 'ALB', 
         'RBC', 'eGFR', 'MPV', 'PA', 'Na+', 'pH', 'HCT', 'GGT', 
         'APOE', 'ALP', 'FO2Hb', 'AG', 'PCT', 'APTT', 'K', 'AB', 
         'LPS', 'proBNP', 'pCO2', 'ALT', 'A/G', 'LDL', 'Lp(a)', 
         'RDW-SD', 'MONO%', 'NEUT%', 'TnT', 'LYMPH%', 'BASO%', 
         'procalcitonin', 'P', 'HHb', 'AST', 'IDBIL', 'Ca2+', 
         'T', 'TT']

# 3 get the train, valid, test data

In [5]:
data = np.load('../input/sample-xjtu/dic_3_1.npy',allow_pickle = True)
data1 = []
for i in data:
    if np.sum(i[3][:,index]) != 0:
        x = torch.FloatTensor(i[2][:, index]).to(device)
        y = torch.FloatTensor([i[-1]]).to(device)
        data1.append((x, y))
data = data1
print(len(data))

7878


In [6]:
def get_data_min_max(records):
    data_min, data_max = None, None
    inf = torch.Tensor([float("Inf")])[0].to(device)
    for b, (vals, labels) in enumerate(records):
        
        batch_min = torch.min(vals, dim = 0)[0]
        batch_max = torch.max(vals, dim = 0)[0]
        if (data_min is None) and (data_max is None):
            data_min = batch_min
            data_max = batch_max
        else:
            data_min = torch.min(data_min, batch_min)
            data_max = torch.max(data_max, batch_max)
    return data_min, data_max

def data_norm(data):
    data_new = []
    data_min, data_max = get_data_min_max(data)
    zero_index = torch.where(data_min==data_max)[0]
    if zero_index.size(0) != 0:
        data_max[zero_index] = 1
    for (vals,label) in data:
        data_new.append(((vals-data_min)/(data_max-data_min), label))
    return data_new

data = data_norm(data)

In [7]:
trainset,testset = model_selection.train_test_split(data,test_size = 0.2, random_state = 42, shuffle = True)
trainset,validset = model_selection.train_test_split(trainset, test_size = 0.125, random_state = 42)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=50, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=False)

In [8]:
def check_label(data):
    n = 0
    for (i,j) in data:
        n = n + j
    print('1:',n,'\t0:',len(data)-n,'\ttotal:',len(data))

check_label(data)
check_label(trainset)
check_label(validset)
check_label(testset)
auc_value = []
result_all = []

1: tensor([3051.], device='cuda:0') 	0: tensor([4827.], device='cuda:0') 	total: 7878
1: tensor([2115.], device='cuda:0') 	0: tensor([3399.], device='cuda:0') 	total: 5514
1: tensor([332.], device='cuda:0') 	0: tensor([456.], device='cuda:0') 	total: 788
1: tensor([604.], device='cuda:0') 	0: tensor([972.], device='cuda:0') 	total: 1576


# 4 Define train and test function

In [9]:
def train(epoch, dataloader, name):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    predicted_all = torch.ones(0).to(device)
    results = torch.ones(0).to(device)
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = net(torch.squeeze(inputs, 1))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        # _, predicted = outputs.max(1)
        predicted = torch.round(outputs)
        predicted_all = torch.cat((predicted_all,predicted))
        results = torch.cat((results,targets))
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    model_auc = roc_auc_score(results.cpu().detach().numpy(),predicted_all.cpu().detach().numpy())
    model_save(net, name, epoch)
    print('train:',batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total),model_auc)
    return model_auc

In [10]:
def test(epoch, dataloader):
    result =[]
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    predicted_all = torch.ones(0).to(device)
    results = torch.ones(0).to(device)
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            outputs = net(torch.squeeze(inputs, 1))
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            # _, predicted = outputs.max(1)
            predicted = torch.round(outputs)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            predicted_all = torch.cat((predicted_all,outputs))
            results = torch.cat((results,targets))
        model_auc = roc_auc_score(results.cpu().detach().numpy(),predicted_all.cpu().detach().numpy())
        print('test:',batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total), model_auc)
        return model_auc,(results.cpu().detach().numpy().reshape(-1),predicted_all.cpu().detach().numpy().reshape(-1))

In [11]:
def model_save(model, name, epoch):
    if not os.path.exists('./' + name):
        os.makedirs('./' + name)
    torch.save(model, './' + name + '/' + str(epoch) + '.pt')

def best_result(result):
    best_index = np.where(result[:,0] == result[:,0].max())
    print('best roc_auc on validset:', best_index[0][0], result[:,0].max())
    return best_index[0][0]

In [12]:
result = {}

# 5 RNN network

In [13]:
class RNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_classes):
        super(RNN,self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(in_dim, hidden_dim, n_layer, batch_first = True)
        self.classifier = nn.Sequential(nn.Linear(hidden_dim, 1), nn.Sigmoid())
    def forward(self, x):
        out, h_n = self.rnn(x)
        x = h_n[-1,:,:]
        x = self.classifier(x)
        return x

In [14]:
net = RNN(99, 50, 2, 1)
net.to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
print(net)

RNN(
  (rnn): RNN(99, 50, num_layers=2, batch_first=True)
  (classifier): Sequential(
    (0): Linear(in_features=50, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [15]:
start = time.process_time()
result_valid = []
for epoch in range(100):
    train(epoch, trainloader, 'RNN')
    result_valid.append(np.array(test(epoch,validloader)))
result_valid = np.array(result_valid)
end = time.process_time()
print('cost time:', end-start)

best_index = best_result(result_valid)
net = torch.load('./RNN/' + str(str(best_index)) +'.pt')
torch.save(net, 'best_RNN.pt')
auc, result_rnn = test(best_index,testloader)
result['RNN'] = result_rnn


Epoch: 0
train: 110 111 Loss: 0.664 | Acc: 61.425% (3387/5514) 0.4990385157086251
test: 15 32 Loss: 0.671 | Acc: 57.868% (456/788) 0.7361881737476221

Epoch: 1
train: 110 111 Loss: 0.633 | Acc: 64.436% (3553/5514) 0.5481948174160527
test: 15 32 Loss: 0.607 | Acc: 68.020% (536/788) 0.756096755442824

Epoch: 2
train: 110 111 Loss: 0.580 | Acc: 69.568% (3836/5514) 0.63938872300781
test: 15 32 Loss: 0.547 | Acc: 72.843% (574/788) 0.8160206615937434

Epoch: 3
train: 110 111 Loss: 0.532 | Acc: 73.540% (4055/5514) 0.6949125629356987
test: 15 32 Loss: 0.490 | Acc: 78.934% (622/788) 0.8626215387867259

Epoch: 4
train: 110 111 Loss: 0.500 | Acc: 75.861% (4183/5514) 0.7240116513200587
test: 15 32 Loss: 0.519 | Acc: 75.381% (594/788) 0.8820016909744239

Epoch: 5
train: 110 111 Loss: 0.494 | Acc: 76.260% (4205/5514) 0.732338255515285
test: 15 32 Loss: 0.456 | Acc: 79.442% (626/788) 0.8858526210103572

Epoch: 6
train: 110 111 Loss: 0.453 | Acc: 78.981% (4355/5514) 0.7603869723886251
test: 15 32 Los

# 6 LSTM network

In [16]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_classes):
        super(LSTM,self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first = True)
        self.classifier = nn.Sequential(nn.Linear(hidden_dim, 1), nn.Sigmoid())

    def forward(self, x):
        out, (h_n, c_n) = self.lstm(x)
        x = h_n[-1, :, :]
        x = self.classifier(x)
        return x

In [17]:
net = LSTM(99, 50, 2, 1)
net.to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
print(net)

LSTM(
  (lstm): LSTM(99, 50, num_layers=2, batch_first=True)
  (classifier): Sequential(
    (0): Linear(in_features=50, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [18]:
start = time.process_time()
result_valid = []
for epoch in range(100):
    train(epoch, trainloader, 'LSTM')
    result_valid.append(np.array(test(epoch,validloader)))
result_valid = np.array(result_valid)
end = time.process_time()
print('cost time:', end-start)

best_index = best_result(result_valid)
net = torch.load('./LSTM/' + str(str(best_index)) +'.pt')
torch.save(net, 'best_LSTM.pt')
auc, result_lstm = test(best_index,testloader)
result['LSTM'] = result_lstm


Epoch: 0
train: 110 111 Loss: 0.673 | Acc: 58.778% (3241/5514) 0.5011380067979945
test: 15 32 Loss: 0.685 | Acc: 57.868% (456/788) 0.518574297188755

Epoch: 1
train: 110 111 Loss: 0.665 | Acc: 61.643% (3399/5514) 0.5
test: 15 32 Loss: 0.684 | Acc: 57.868% (456/788) 0.5758395423800465

Epoch: 2
train: 110 111 Loss: 0.665 | Acc: 61.643% (3399/5514) 0.5
test: 15 32 Loss: 0.683 | Acc: 57.868% (456/788) 0.6319917829211583

Epoch: 3
train: 110 111 Loss: 0.666 | Acc: 61.643% (3399/5514) 0.5
test: 15 32 Loss: 0.682 | Acc: 57.868% (456/788) 0.6802109754808708

Epoch: 4
train: 110 111 Loss: 0.664 | Acc: 61.643% (3399/5514) 0.5
test: 15 32 Loss: 0.682 | Acc: 57.868% (456/788) 0.7040431462692877

Epoch: 5
train: 110 111 Loss: 0.665 | Acc: 61.643% (3399/5514) 0.5
test: 15 32 Loss: 0.682 | Acc: 57.868% (456/788) 0.7139842263792009

Epoch: 6
train: 110 111 Loss: 0.664 | Acc: 61.643% (3399/5514) 0.5
test: 15 32 Loss: 0.682 | Acc: 57.868% (456/788) 0.7173166349608964

Epoch: 7
train: 110 111 Loss: 0.6

In [19]:
for key, value in result.items():
    print(key,':',roc_auc_score(value[0],value[1]))
np.save('result.npy', result)

RNN : 0.92383765295833
LSTM : 0.9282645872509743
